# Four point functions in the $O(n)$ model

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [ ]:
cd("/home/roux/PottsOn/bootstrap/torus1pt/code")
include("sphere4pt.jl")
println("Number of threads: ", Threads.nthreads())

[ Info: Precompiling BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c] (cache misses: include_dependency fsize change (4), wrong source (2))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


Number of threads: 8


# Bootstrap equations

We solve crossing symmetry equations for the spectrum of the $O(n)$ CFT:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
\end{align}
We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(1-x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(1/x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

# Some numerical four-point functions

## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [2]:
ind = ((1//2, 0), (1//2, 0), (1//2, 0), (1//2, 0))
blocks = precompute_blocks(ind, parity=0, precision=20);

### Signature $(0, 1, 1)$

In [3]:
sol = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false), rmax=2);

 Channel s
┌─────────────────┬───────────────────────────┬───────────┐
│ Field           │ Structure constant        │ Rel. err. │
├─────────────────┼───────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                 │ 0         │
│ (1, 0)          │ -0.167173+0.134319im      │ 4.6e-18   │
│ (2, 0)          │ -0.00239852+0.000773346im │ 2e-16     │
│ (2, 1)          │ 0.000797319-0.000705013im │ 1.2e-16   │
└─────────────────┴───────────────────────────┴───────────┘
 Channel t
┌────────────┬───────────────────────────┬───────────┐
│ Field      │ Structure constant        │ Rel. err. │
├────────────┼───────────────────────────┼───────────┤
│ (1, 0)     │ 0.422102+0.0893706im      │ 1.8e-18   │
│ (1, 1)     │ 0.16568-0.0434621im       │ 7.3e-19   │
│ (2, 0)     │ 0.000999679-1.2823e-05im  │ 4.2e-16   │
│ (2, -1//2) │ 0.000592214+0.000185033im │ 9.5e-16   │
│ (2, 1//2)  │ 0.000592214+0.000185033im │ 9.9e-16   │
│ (2, 1)     │ 0.000344058+0.000230614im │ 6.9e-16   │
└──

In [42]:
BootstrapVirasoro.find_normalised(sol.str_cst)

(P = (0.2+0.1im), :s)

Benchmarks: on my 2015, 4-core Intel Macbook pro.
| Precision          | Python | Julia |
|----------|----------|----------|
| $\Delta_{\mathrm{max}}=20$, 13 digits  | 2min23s  | 2.3s  |
| $\Delta_{\mathrm{max}}=30$, 25 digits  | 8min10s  | 3.0s  |
| $\Delta_{\mathrm{max}}=40$, 35 digits  | 23min18s  | 12.6s  |

### Signature $(0, 1, 0)$, $D^{(s)}_{(2, 0)} = 0$

In [5]:
fix = [(:s, diag_field, 1), (:s, Field(c, r=2, s=0), 0)]
solve(blocks, Sig(0, 1, 0), fix=fix, rmax=2);

 Channel s
┌─────────────────┬──────────────────────────┬────────────────┐
│ Field           │ Structure constant       │ Relative error │
├─────────────────┼──────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                1.0+0.0im │            0.0 │
│          (1, 0) │      -1.16517-0.591022im │    7.65818e-15 │
│          (1, 1) │     0.464566+0.0798147im │     8.4386e-15 │
│          (2, 0) │                0.0+0.0im │            0.0 │
│       (2, 1//2) │ -0.00318409+0.00145081im │    1.01034e-14 │
│      (2, -1//2) │ -0.00318409+0.00145081im │     8.2106e-15 │
│          (2, 1) │  0.00257332-0.00274978im │    6.86828e-15 │
└─────────────────┴──────────────────────────┴────────────────┘
 Channel t
┌────────────┬──────────────────────────┬────────────────┐
│ Field      │ Structure constant       │ Relative error │
├────────────┼──────────────────────────┼────────────────┤
│     (1, 0) │     -0.433576-0.577531im │    1.38597e-14 │
│     (1, 1) │     0.672446+0.0319329i

### Solve for many different Ps

In [ ]:
# solve for many different values of the momentum for the diag field in the channel.
# only recompute one column of the linear system each time: very cheap.
Ps = [big"0.8" + big"0.2"*im + i / big(100)*im for i in 1:3];
diagblocks = compute_diagblocks(blocks, Ps, :s)
sol_Ps = solve(blocks, Sig(0, 1, 1), even_spin=Channels(true, false, false),  Ps=Ps, diagchan=:s)
# print one of the results
show(stdout, sol_Ps[2].str_cst, rmax=2)

## $\left\langle P_1 P_2 P_3 P_4 \right\rangle$

In [7]:
ss = rand(Complex{BigFloat}, 4)
ind = Tuple((0, s) for s in ss)
blocks = precompute_blocks(ind, fields, parity=1, precision=15);

In [8]:
sol = solve(blocks, Sig(0, 0, 0), show=true);

 Channel s
┌─────────────────┬────────────────────────────┬────────────────┐
│ Field           │ Structure constant         │ Relative error │
├─────────────────┼────────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │                  1.0+0.0im │            0.0 │
│          (1, 0) │      -0.043652+0.0461863im │    6.68646e-10 │
│          (1, 1) │     -0.00502903+0.098547im │     1.1386e-10 │
│          (2, 0) │   0.00079105-0.000735744im │     4.28782e-8 │
│       (2, 1//2) │  0.000110591-0.000183974im │     1.46143e-7 │
│          (2, 1) │ -0.000189192-3.69278e-06im │     5.65625e-8 │
│          (3, 0) │                          0 │      0.0607838 │
│       (3, 1//3) │                          0 │      0.0644803 │
│       (3, 2//3) │                          0 │      0.0288652 │
│          (3, 1) │                          0 │      0.0139439 │
└─────────────────┴────────────────────────────┴────────────────┘
 Channel t
┌─────────────────┬───────────────────────────┬───────

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [9]:
ind = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))
blocks = precompute_blocks(ind, fields2, parity=1, precision=13);

UndefVarError: UndefVarError: `fields2` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Signature $(0, \frac32, \frac32)$

In [10]:
sol = solve(blocks, Sig(0, 3//2, 3//2));

 Channel s
┌─────────────────┬────────────────────────┬────────────────┐
│ Field           │ Structure constant     │ Relative error │
├─────────────────┼────────────────────────┼────────────────┤
│ (P=0.10+0.10im) │              1.0+0.0im │            0.0 │
│          (1, 0) │    -1.79924+0.421555im │      0.0536187 │
│          (1, 1) │ 0.000273845+0.203333im │       0.179268 │
│          (2, 0) │      1.05837-2.37073im │       0.276612 │
│       (2, 1//2) │   -0.557097+0.386062im │        1.79628 │
│          (2, 1) │    -0.133632-0.76917im │       0.431461 │
│          (3, 0) │      41.9787-27.2899im │       0.626559 │
│       (3, 1//3) │     -34.9665+26.8837im │       0.680484 │
│       (3, 2//3) │       24.938-25.2727im │       0.667133 │
│          (3, 1) │     -11.1594+19.0543im │       0.716201 │
└─────────────────┴────────────────────────┴────────────────┘
 Channel t
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├─

### Signature $(1, \frac12, \frac32)$

In [11]:
solve(blocks, Sig(1, 1//2, 3//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.314676-0.227065im │        1.58253 │
│    (2, 0) │   -0.756645+5.0774im │       0.595367 │
│ (2, 1//2) │  -0.515941-3.00656im │       0.911785 │
│    (2, 1) │   0.973783+1.64767im │        1.26323 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (1, 0) │ -0.0288501+0.0165005im │          1.011 │
│    (1, 1) │  0.0329273-0.0770832im │       0.777737 │
│    (2, 0) │      3.39492+1.71516im │          2.017 │
│ (2, 1//2) │     -2.78334-1.07935im │        1.75393 │
│    (2, 1) │    1.69856+0.0715347im │        1.71933 │
└───────────┴────────────────────────┴──────

### Signature $(1, \frac32, \frac12)$

In [12]:
solve(blocks, Sig(1, 3//2, 1//2), rmax=2);

 Channel s
┌───────────┬──────────────────────┬────────────────┐
│ Field     │ Structure constant   │ Relative error │
├───────────┼──────────────────────┼────────────────┤
│    (1, 0) │            1.0+0.0im │            0.0 │
│    (1, 1) │ -0.422351-0.268128im │       0.215619 │
│    (2, 0) │    -2.42269+4.0059im │       0.383119 │
│ (2, 1//2) │    1.60603-2.13824im │       0.575963 │
│    (2, 1) │  -0.166614+1.56393im │       0.488147 │
└───────────┴──────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────┬────────────────┐
│ Field     │ Structure constant │ Relative error │
├───────────┼────────────────────┼────────────────┤
│    (2, 0) │  2.02455+1.94047im │       0.673465 │
│ (2, 1//2) │ -1.66763-1.39314im │       0.676533 │
│    (2, 1) │ 1.12007+0.374722im │       0.710549 │
└───────────┴────────────────────┴────────────────┘
 Channel u
┌───────────┬───────────────────────┬────────────────┐
│ Field     │ Structure constant    │ Relative error │
├──────

## $\left\langle (0, \frac12)^3 P_{(1, 0)} \right\rangle$

In [32]:
c = CC(β=1 / (big"0.8" + big"0.2" * im))
ind = ((0, 1//2), (0, c.β^2), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, parity=1, precision=20);
sol_diag = solve(blocks, Sig(0, 0, 0), even_spin=Channels(true), rmax=3, rels=:stu);

Channels s, t, u:
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ P = (0.2+0.1im) │ 1.0+0.0im                  │ 0         │
│ (1, 0)          │ 1.07293+0.534094im         │ 1.5e-21   │
│ (2, 0)          │ -0.00423291+0.00317678im   │ 1.2e-19   │
│ (2, 1)          │ 0.00274774+0.000554227im   │ 7.6e-20   │
│ (3, 0)          │ -1.79936e-07+2.23242e-07im │ 6.7e-16   │
│ (3, 2//3)       │ -1.4615e-07-9.52926e-07im  │ 1.4e-16   │
└─────────────────┴────────────────────────────┴───────────┘


## $\left\langle (0, \frac12)^3 (1, 0) \right\rangle$

In [28]:
ind = ((0, 1//2), (1, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, parity=1, precision=20)
sol = solve(blocks, Channels(0, 1, 1), show=false, even_spin=Channels(true, false, false));

## $\left\langle (0, \frac12)^3 (2, 0) \right\rangle$

In [16]:
ind = ((0, 1//2), (2, 0), (0, 1//2), (0, 1//2))
blocks = precompute_blocks(ind, fields, parity=1, precision=20);
solve(blocks, Sig(1, 2, 1), even_spin=Channels(false, true, false), rmax=3);

 Channel s
┌───────────┬────────────────────────────┬────────────────┐
│ Field     │ Structure constant         │ Relative error │
├───────────┼────────────────────────────┼────────────────┤
│    (1, 0) │                  1.0+0.0im │            0.0 │
│    (1, 1) │        -0.63854+0.181026im │    2.98082e-17 │
│    (2, 0) │                          0 │       0.880898 │
│ (2, 1//2) │                          0 │       0.970237 │
│    (2, 1) │                          0 │        1.06903 │
│    (3, 0) │ -7.86521e-05-0.000322813im │    9.03894e-12 │
│ (3, 1//3) │   4.37238e-05+0.00014278im │    1.86741e-11 │
│ (3, 2//3) │  4.92031e-05+9.64753e-05im │    1.78644e-11 │
│    (3, 1) │ -8.77471e-05-9.14424e-05im │    9.44656e-12 │
└───────────┴────────────────────────────┴────────────────┘
 Channel t
┌───────────┬────────────────────────┬────────────────┐
│ Field     │ Structure constant     │ Relative error │
├───────────┼────────────────────────┼────────────────┤
│    (2, 0) │    0.413186+0.31

## $\left\langle (0, \frac12)^3 (3, 0) \right\rangle$

# Polynomials

In [50]:
nrange(length) = range(big"2.065" + big"1.3588"*im, big"2.6247" - big"2.13339"*im , length)
wrange(length) = range(-big"0.868", big"0.932", length);

function print_polynomials(io::IO, polys, degs, varnames)
    for (r, s) in sort(collect(keys(polys)))
        fit_poly = fit(polys[r, s], varnames, degs[r])
        indices = replace(sprint(print, "$r, $s"), "//" => "/")
        print(io, "d[$indices] = ")
        show(io, fit_poly, cutoff=1e-3, digits=16)
        println(io, ";")
    end
end;

## One-point checks

In [2]:
include("torus1pt.jl");
@load "results/V10.jld2"

2-element Vector{Symbol}:
 :data
 :Dnorms

In [7]:
polys = reduce_to_poly(data, 3, 0, Dnorms);

In [12]:
show(stdout, fit(polys, (:w1, :w, :n), (0, 5, 6)), cutoff=1e-6)

-1.3333 * n^2 + 30.667 * w*n^2 + 5.3333 * w^2*n^2 + -26.667 * w^3*n^2 + 5.3333 * w^5*n^2 + 5.3333 * n^3 + -13.333 * w*n^4 + -2.6667 * w^2*n^4 + 12 * w^3*n^4 + -2.6667 * w^5*n^4 + -1.3333 * n^5 + 0.33333 * n^6 + 1.6667 * w*n^6 + 0.33333 * w^2*n^6 + -1.3333 * w^3*n^6 + 0.33333 * w^5*n^6

## $\langle V_{(1, 0)} \rangle$

In [ ]:
ns = nrange(12)
ws = wrange(12)
ext_inds = [((0, 1 // 2), (1, 0), (0, 1 // 2), (0, 1 // 2))]
sig = Channels(0, 1, 1)

data, Dnorms = compute_bootstrap_data(ext_inds, sig, ns, ws, parity=1, prec=25, even_spin=Channels(true, false, false));
# @save "results/V10_sphere.jld2" data Dnorms
# @load "results/V10_sphere.jld2"

In [3]:
@load "results/V10_sphere.jld2"

1-element Vector{Symbol}:
 :polys

In [48]:
import BootstrapVirasoro.LoopModels.κrs
new_polys = @channels Dict()
for chan in (:s, :t, :u)
    for (r, s) in keys(polys[chan])
        poly = polys[chan][r, s]
        new_polys[chan][r, s] =  [
            ((poly[i][1][1], poly[i][1][2], poly[i][1][3]^2), # (w1, w, n^2)
             poly[i][2] * κrs(r, s, βfromn(poly[i][1][3])^2))
            for i in eachindex(poly)
        ]
    end
end

In [ ]:
# polys = @channels reduce_to_poly(data, Dnorms, chan, 1:3, r->0:(2//r):0);
# polys_t = reduce_to_poly(data, Dnorms, :t, 1:1:3, r->0:(1//r):1);
# @save "results/V10polys.jld2" polys_s polys_t
# @load "results/V10polys.jld2"
degs = Dict(1 => (0, 0, 0), 2 => (0, 1, 2), 3 => (0, 4, 6), 4 => (0, 3, 6), 5 => (0, 8, 11))
# open("results/V10polys.m", "w") do io
let io = stdout;
    # print_polynomials(io, polys_s, degs)
    print_polynomials(io, new_polys.t, degs, (:w1, :w, :n2))
end

In [52]:
n = ns[1]
w = ws[1]
c = cfromn(n)
DG = DoubleGamma(c.β)
P = acos(w/2) / 2 / π / c.β
VP = Field(c, P=P)

Vs = Tuple(Field(c, r=r, s=s) for (r, s) in ext_inds[1])
V = Field(c, r=3, s=2//3)
Dref = compute_reference(Vs, V, :s, DG)
Dnorm = compute_reference(Vs, VP, :s, DG)

d = 2 // 3 * (4 - w^2 + (2 - n^2) * (w^2 - 2))
Dref / Dnorm * d * 16

7.891534827148175417510695883e-05 + 6.681817727885003502490150575e-06im

In [54]:
data[1][2].t.constants[Field(c, r=3, s=2//3)]

8.067932562594766924298262372e-05 - 4.818369250086839511990457855e-06im

## $\langle V_{(2, 0)} \rangle$

In [ ]:
ns = nrange(6)
ws = wrange(6)
ext_inds = [((0, 1 // 2), (2, 0), (0, 1 // 2), (0, 1 // 2))]
sig = Channels(0, 2, 2)

data, Dnorms = compute_bootstrap_data(ext_inds, sig, ns, ws, parity=1, prec=25, even_spin=Channels(true, false, false));
@save "results/V20_sphere.jld2" data Dnorms
# @load "results/V10_sphere.jld2" 

In [68]:
polys_s = reduce_to_poly(data, Dnorms, :s, 1:1:4, r->0:(2//r):1)
polys_t = reduce_to_poly(data, Dnorms, :t, 2:1:4, r->0:(2//r):1)

Dict{Tuple{Int64, Rational{Int64}}, Vector{Any}} with 7 entries:
  (4, 0)    => [((2.0, -0.868, 2.065+1.3588im), -0.0424566-0.0412686im), ((2.0,…
  (4, 1//2) => [((2.0, -0.868, 2.065+1.3588im), -0.034121+0.044303im), ((2.0, -…
  (2, 0)    => [((2.0, -0.868, 2.065+1.3588im), 0.25-6.32691e-14im), ((2.0, -0.…
  (3, 2//3) => [((2.0, -0.868, 2.065+1.3588im), -0.144667+3.66852e-14im), ((2.0…
  (4, 1)    => [((2.0, -0.868, 2.065+1.3588im), -0.0424566-0.0412686im), ((2.0,…
  (2, 1)    => [((2.0, -0.868, 2.065+1.3588im), 0.25-6.32691e-14im), ((2.0, -0.…
  (3, 0)    => [((2.0, -0.868, 2.065+1.3588im), -0.144667+3.66852e-14im), ((2.0…

In [74]:
degs = Dict(1 => (0, 0, 0), 2 => (0, 0, 0), 3 => (0, 1, 1), 4 => (0, 2, 4))
let io = stdout;
    print_polynomials(io, polys_s, degs)
    print_polynomials(io, polys_t, degs)
end

d[1, 0/1] = ;
d[2, 0/1] = ;
d[2, 1/1] = ;
d[3, 0/1] = ;
d[3, 2/3] = ;
d[4, 0/1] = 0.2500000000025658;
d[4, 1/2] = -0.2500000000025315;
d[4, 1/1] = 0.2500000000025655;
d[2, 0/1] = 0.2499999999999506;
d[2, 1/1] = 0.2499999999999506;
d[3, 0/1] = 0.1666666666667148 * w;
d[3, 2/3] = 0.1666666666667148 * w;
d[4, 0/1] = 2.686750332344111-0.6948624591792011I + 0.1250000000012652 * w^2 + -3.421687495025936+0.7842535717715033I * n + 1.728665456287835-0.3035301224273246I * n^2 + -0.4222178204298912+0.03817231341146941I * n^3 + 0.04204090160093148+0.001744038516574339I * n^4;
d[4, 1/2] = -1.215522569352468-0.009669988862660154I + 0.1250000000012652 * w^2 + 1.20936879097531+0.07940658813420413I * n + -0.5623431118542779-0.08458336570138648I * n^2 + 0.1256508933945444+0.03403727384254487I * n^3 + -0.01109470678456436-0.00502271582117542I * n^4;
d[4, 1/1] = 2.686750332344111-0.6948624591792012I + 0.1250000000012652 * w^2 + -3.421687495025936+0.7842535717715034I * n + 1.728665456287835-0.3035301224273

## $\langle V_{(3, \frac23)} \rangle$

In [ ]:
ns = nrange(6)
ws = wrange(6)
βs = [sqrt(acos(-n/2)/π) for n in ns]
ext_inds = [((0, 1 // 2), (3, 2 // 3), (0, 1 // 2), (0, 1 // 2)) for β in βs]

sig = Channels(0, 3, 3)
data, Dnorms = compute_bootstrap_data(ext_inds, sig, ns, ws, parity=1, prec=40, even_spin=Channels(true, false, false));
@save "results/V3_23_sig_033_sphere.jld2" data, Dnorms

In [ ]:
sig = Channels(1, 3, 2)
data, Dnorms = compute_bootstrap_data(ext_inds, sig, ns, ws, parity=1, prec=40, even_spin=Channels(true, false, false));
@save "results/V3_23_sig_132_sphere.jld2" data, Dnorms
# @load "results/V10_sphere.jld2" 